# Qcodes example with Keithley 2600

**NOTE** This example is for the new (as of September 2017) channelised driver. If you are considering using the old driver, please reconsider. There is no functionality of the old driver that is not contained in the new one, but it might have a (slightly) different name.

In [1]:
import qcodes as qc

from qcodes.instrument_drivers.tektronix.Keithley_2600_channels import Keithley_2600

In [2]:
# Create a station to hold all the instruments

station = qc.Station()

# instantiate the Keithley and add it to the station

keith = Keithley_2600('keithley', 'TCPIP0::192.168.15.116::inst0::INSTR')
station.add_component(keith)

Connected to: Keithley Instruments Inc. 2614B (serial:4084407, firmware:3.2.1) in 0.09s


'keithley'

The Keithley 2600 has two channels, here called `smua` and `smub` in agreement with the instrument manual.

The two channels are basically two separate instruments with different integration times (`nplc`), operation modes (`mode`) etc.

In [3]:
# Get an overview of the settings
#
# You will notice that the two channels have identical parameters but
# potentially different values for them
#
keith.print_readable_snapshot()

keithley:
	parameter      value
--------------------------------------------------------------------------------
IDN             :	{'vendor': 'Keithley Instruments Inc.', 'model': '2614B', '...
display_settext :	None 
timeout         :	5 (s)
keithley_smua:
	parameter     value
--------------------------------------------------------------------------------
curr           :	3.0994e-07 (A)
fastsweep      :	None 
limiti         :	0.1 (A)
limitv         :	20 (V)
measurerange_i :	0.1 (A)
measurerange_v :	2.00000e+01 (V)
mode           :	voltage 
nplc           :	0.05 
output         :	on 
sourcerange_i  :	1e-07 (A)
sourcerange_v  :	20 (V)
volt           :	1.9999 (V)
keithley_smub:
	parameter     value
--------------------------------------------------------------------------------
curr           :	-3.6955e-13 (A)
fastsweep      :	None 
limiti         :	0.1 (A)
limitv         :	20 (V)
measurerange_i :	1e-07 (A)
measurerange_v :	2.00000e-01 (V)
mode           :	voltage 
nplc           :	1 
ou

## Basic operation

Each channel operates in either `voltage` or `current` mode. The mode controls the _source_ behaviour of the instrument, i.e. `voltage` mode corresponds to an amp-meter (voltage source, current meter) and vice versa.

In [4]:
# Let's set up a single-shot current measurement
# on channel a

keith.smua.mode('voltage')
keith.smua.nplc(0.05)  # 0.05 Power Line Cycles per measurement. At 50 Hz, this corresponds to 1 ms
keith.smua.sourcerange_v(20) 
keith.smua.measurerange_i(0.1)
#
keith.smua.volt(1)  # set the source to output 1 V
keith.smua.output('on')  # turn output on
curr = keith.smua.curr()
keith.smua.output('off')

print('Measured one current value: {} A'.format(curr))

Measured one current value: -1.54972e-06 A


## Fast IV curves

Onboard the Keithley 2600 sits a small computer that can interpret `Lua` scripts. This can be used to make fast IV-curves and is supported by the QCoDeS driver.

In [5]:
# Let's make a fast IV curve by sweeping the voltage from 1 V to 2 V in 500 steps
# (when making this notebook, nothing was connected to the instrument, so we just measure noise)

In [6]:
# This function performs the fast sweep and returns a QCoDeS DataSet
data = keith.smua.doFastSweep(1, 2, 500, mode='IV')

DataSet:
   location = 'data/2017-09-28/#001_{name}_14-43-10'
   <Type>   | <array_id>             | <array.name> | <array.shape>
   Measured | keithley_smua_iv_sweep | iv_sweep     | (500,)
acquired at 2017-09-28 14:43:11


In [7]:
# The DataSet may be plotted
plot = qc.MatPlot()
plot.add(data.arrays['keithley_smua_iv_sweep'])

In [8]:
# Finally, tear down the instrument
keith.close()